In [122]:
#--Dependencies--#

#-- Data Cleaning Libraries:
import pandas as pd
import numpy as np

#-- Data Visualization Libraries:
from matplotlib import pyplot as plt
import seaborn as sns #--just in case we need it, probably won't


#-- Web Scraping Libraries:
import os
import time
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser

In [123]:
#Read CSV
raw_df = pd.read_csv("Players_birth_raw.csv")


In [124]:
raw_df.head()

,1-Jan
0,Bones McKinney (1919)
1,John Logan (1921)
2,Ray Wertis (1922)
3,Ray Corley (1928)
4,Alan Sawyer (1928)


In [125]:
#Create function to take out year

def year_func(row):
    year = row[-5: -1]
    
    return year

In [126]:
raw_df["Year"]= raw_df["1-Jan"].apply(lambda x : year_func(x))
raw_df.head()

,1-Jan,Year
0,Bones McKinney (1919),1919
1,John Logan (1921),1921
2,Ray Wertis (1922),1922
3,Ray Corley (1928),1928
4,Alan Sawyer (1928),1928


In [127]:
#normalize year

def name_func(row):
    name = row[0:-6]
    
    return name

In [128]:
raw_df["1-Jan"] = raw_df["1-Jan"].apply(lambda row: name_func(row))


In [129]:
raw_df.head()

,1-Jan,Year
0,Bones McKinney,1919
1,John Logan,1921
2,Ray Wertis,1922
3,Ray Corley,1928
4,Alan Sawyer,1928


In [130]:
raw_df = raw_df.rename(columns={"1-Jan": "Player"})


In [131]:
raw_df = raw_df[raw_df.Player != '']

In [132]:
#Player index function

def play_index(row):
    player = row.Player
    year = row.year
    
    index = player + "_" + year
    
    return index

In [133]:
raw_df["Player Index"] = raw_df.Player + "_" + raw_df.Year

In [134]:
raw_df = raw_df.sort_values(by= "Year")

In [135]:
raw_df = raw_df.reset_index(drop=True)
raw_df.head()

,Player,Year,Player Index
0,Nat Hickey,1902,Nat Hickey _1902
1,Charley Shipp,1913,Charley Shipp _1913
2,Ben Goldfaden,1913,Ben Goldfaden _1913
3,Joe Fabel,1913,Joe Fabel _1913
4,Nat Frankel,1913,Nat Frankel _1913


In [136]:
raw_df["Year"] = raw_df["Year"].apply(lambda x: int(x))

In [137]:
raw_df = raw_df[raw_df.Year >= 1980]
raw_df.head()

,Player,Year,Player Index
3358,Tayshaun Prince,1980,Tayshaun Prince _1980
3359,Adam Harrington,1980,Adam Harrington _1980
3360,Juan Carlos Navarro,1980,Juan Carlos Navarro _1980
3361,Theron Smith,1980,Theron Smith _1980
3362,Damien Wilkins,1980,Damien Wilkins _1980


In [138]:
raw_df.Player = raw_df.Player.apply(lambda x: x.strip())

In [139]:
raw_df[raw_df.Player == "LeBron James"]

,Player,Year,Player Index
3682,LeBron James,1984,LeBron James _1984


In [140]:
pindex_df = raw_df[["Player", "Player Index"]]

In [141]:
ncaa_df = pd.read_csv("NCAA_clean.csv", header=0)

In [142]:
ncaa_df = ncaa_df.drop(columns= "Unnamed: 0", index=1)

In [144]:
ncaa_df.head()

,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
0,Alaa Abdelnaby,SR,1980,F,Duke,ACC,38,24.921,5.711,9.211,...,0.0,0.0,6.632,0.711,0.684,1.263,1.421,2.605,15.053,25.92
2,Doug Able,JR,1980,F,Boston College,Big East,27,30.074,4.667,9.185,...,0.0,0.0,7.370,0.926,1.000,0.852,2.593,2.963,12.037,14.96
3,Frank Ableson,NaN,1980,G,Cornell,Ivy,28,12.750,1.357,3.071,...,0.0,0.0,1.286,0.607,0.321,0.000,1.357,1.214,4.214,10.30
4,Henri Abrams,JR,1980,C,George Mason,CAA,17,4.294,0.353,0.765,...,0.0,0.0,1.353,0.000,0.118,0.647,0.471,0.824,0.941,7.63
5,Rick Achberger,JR,1980,F,Stephen F. Austin,Southland,27,19.407,3.889,6.926,...,0.0,0.0,4.370,0.741,0.333,0.333,1.889,2.037,8.815,15.43


In [145]:
ncaa_df[ncaa_df["Player"] == "Grayson Allen"]

,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
68867,Grayson Allen,FR,2015,G,Duke,ACC,35,9.200,1.286,3.029,...,0.257,0.743,1.000,0.400,0.286,0.143,0.429,1.200,4.371,22.56
73623,Grayson Allen,SO,2016,G,Duke,ACC,36,36.583,6.639,14.250,...,0.667,3.917,4.583,3.528,1.306,0.139,2.000,2.722,21.639,29.88
78400,Grayson Allen,JR,2017,G,Duke,ACC,34,29.618,4.147,10.500,...,0.676,3.059,3.735,3.500,0.765,0.059,2.176,2.294,14.471,24.64
83205,Grayson Allen,SR,2018,G,Duke,ACC,37,35.649,4.892,11.703,...,0.541,2.784,3.324,4.649,1.730,0.054,2.135,2.135,15.459,24.11


In [146]:
blank_test = pd.merge(pindex_df, ncaa_df, on= "Player" ,how= "outer")

In [147]:
blank_test.columns

Index(['Player', 'Player Index', 'Class', 'Season', 'Pos', 'School', 'Conf',
       'G', 'MP', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 'FT', 'FTA', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER'],
      dtype='object')

In [148]:
blank_test["Team Index"] = blank_test["School"] + "_" + blank_test["Conf"]


In [149]:
#Reorder columns

cols = list(blank_test.columns)

cols.insert(0, cols.pop(cols.index('Player Index')))

In [150]:
cols.insert(1, cols.pop(cols.index('Team Index')))

In [151]:
cols

['Player Index',
 'Team Index',
 'Player',
 'Class',
 'Season',
 'Pos',
 'School',
 'Conf',
 'G',
 'MP',
 'FG',
 'FGA',
 '2P',
 '2PA',
 '3P',
 '3PA',
 'FT',
 'FTA',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'PER']

In [152]:
blank_test = blank_test.reindex(columns= cols)

In [159]:
test_team_na = blank_test[blank_test["Team Index"].isna()]

test_team_na.head()

,Player Index,Team Index,Player,Class,Season,Pos,School,Conf,G,MP,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
7,Juan Carlos Navarro _1980,NaN,Juan Carlos Navarro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Jerome Beasley _1980,NaN,Jerome Beasley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,Raul Lopez _1980,NaN,Raul Lopez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Luis Scola _1980,NaN,Luis Scola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,Leon Smith _1980,NaN,Leon Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
test_team_notna = blank_test[blank_test["Team Index"].notna()]
test_team_notna.head()

,Player Index,Team Index,Player,Class,Season,Pos,School,Conf,G,MP,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
0,Tayshaun Prince _1980,Kentucky_SEC,Tayshaun Prince,FR,1999.0,F,Kentucky,SEC,37.0,20.189,...,0.000,0.000,3.838,1.216,0.676,0.568,1.027,0.919,5.784,14.06
1,Tayshaun Prince _1980,Kentucky_SEC,Tayshaun Prince,SO,1990.0,F,Kentucky,SEC,32.0,33.969,...,0.000,0.000,6.000,1.812,0.844,1.281,2.000,1.688,13.312,17.89
2,Tayshaun Prince _1980,Kentucky_SEC,Tayshaun Prince,JR,2001.0,F,Kentucky,SEC,34.0,32.853,...,1.647,4.853,6.500,2.912,0.706,1.088,1.706,1.647,16.912,28.76
3,Tayshaun Prince _1980,Kentucky_SEC,Tayshaun Prince,SR,2002.0,F,Kentucky,SEC,32.0,33.344,...,2.094,4.219,6.312,1.625,1.062,1.344,2.000,1.594,17.500,28.39
4,Adam Harrington _1980,North Carolina State_ACC,Adam Harrington,FR,1999.0,G,North Carolina State,ACC,32.0,24.562,...,0.000,0.000,2.000,1.594,1.219,0.281,0.000,0.000,11.625,26.59


In [ ]:
test_team_notna.to_csv("NCAA")